In [19]:
%pip install sentence_transformers
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

Defaulting to user installation because normal site-packages is not writeable
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
Using cached sentence_transformers-3.3.1-py3-none-any.whl (268 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
df = pd.read_csv("../data/cleaned_job_descriptions.csv")

In [21]:
#confirm that the number of lines is equal to the number of rows in the dataframe
# df = df.drop_duplicates()
# df['Role'].unique()
# roles_to_keep = ['Frontend Web Developer', 'Wireless Network Engineer', 'User Interface Designer', 'Front-End Developer', 'IT Support Specialist', 'Database Developer', 'Social Science Researcher', 'Record Keeper',
#        'Staff Nurse', 'International Tax Consultant', 'Risk Analyst',
#        'Speech Pathologist', 'Regional Sales Director',
#        'Mental Health Counselor', 'Urban Planner',
#        'Strategic Account Manager', 'User Researcher',
#        'Financial Analyst', 'Desktop Support Technician' ]

# df = df[df['Role'].isin(roles_to_keep)]
# df['Role'].unique()
# df.shape

# df

#(1610462, 7)
#shape after filtering to keep only the roles we are interested in
#(86322, 7)
lines = []
for index, row in df.iterrows():
    row_string = " ".join(row.astype(str))  # Convert each value to string and join
    lines.append(row_string)

print(lines[0])


2 to 12 Years BCA Ashgabat Web Developer Frontend Web Developer Frontend Web Developers design and implement user interfaces for websites, ensuring they are visually appealing and user-friendly. They collaborate with designers and backend developers to create seamless web experiences for users. PNC Financial Services Group


In [22]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(lines, show_progress_bar=True)
embeddings.shape

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Ba

(86327, 384)

In [51]:
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
index.is_trained
print(type(index))


import joblib
#move the model and the index to the model folder
# print("type of model", type(model))
# print("type of index", type(index))

# #save using faiss
# faiss.write_index(index, "../model/faiss_index.faiss")
# #save using joblib
# joblib.dump(model, "../model/model.joblib")


<class 'faiss.swigfaiss_avx2.IndexFlatL2'>


In [52]:
k = 4
query = "I am a student looking for a job in the tech industry, I want to do cyber security"
query_embedding = model.encode(query)
query_embedding = query_embedding.reshape(1, -1)

In [53]:
D, I = index.search(query_embedding, k)
indices = I[0]
upper_bound = df.shape[0]

for index1 in indices:
    if index1 < upper_bound:
        print(df.iloc[index1]['Job_Title'], '\n\n')
    else:
        print("No match found")

Systems Administrator 


UI Developer 


UI Developer 


Network Engineer 




In [54]:

print("type of model", type(model))
print("type of index", type(index))

#save using faiss
faiss.write_index(index, "../model/faiss_index.faiss")
#save using joblib
# joblib.dump(model, "../model/model.joblib")





type of model <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
type of index <class 'faiss.swigfaiss_avx2.IndexFlatL2'>


In [35]:
#save the dataframe as well
df.to_pickle("../model/df.pkl")